# Image Processing: Channel edge detection
This notebook contains the detection of the channel edges for experimental images (Section 2.1.2, step (ii)).

## Functions

In [ ]:
import cv2
import numpy as np
from statistics import median
import matplotlib.pyplot as plt
import os
from os.path import join
import pandas as pd

In [ ]:
def find_min_along_vertical_line(gray, pos_x, start_y, length):
    """
    Find the minimum grayscale value along a vertical line in an image.

    Args:
    gray: a 2D numpy array representing the grayscale image
    pos_x: x-coordinate of the starting point of the vertical line
    start_y: y-coordinate of the starting point of the vertical line
    length: length of the vertical line

    Returns:
    min_gray_val: minimum grayscale value found along the vertical line 
    y_min_gray_val: y-coordinate of the corresponding pixel
     """
    
    # create an array containing the y coordinates of the vertical line
    y_line = np.array(range(start_y, start_y + length))
    
    # initialize the list containing the greyscale values
    gray_vals = []
    
    # loop over y values of the vertical line and get corresponding grayscale values
    for y in y_line:
        gray_val = gray[y, pos_x]
        gray_vals.append(gray_val)

    # Find the minimum grayscale value and corresponding y position
    min_gray_val = min(gray_vals)
    y_min_gray_val = y_line[gray_vals.index(min(gray_vals))]

    return min_gray_val, y_min_gray_val

## (ii) Channel edge detection

**User input: selection of cases and paths**

In [ ]:
# fluid
fluid = "01_water" #available: 01_water, 02_tween, 03_novec, 04_novec_sim*

# paths
path_data_experiments = "../data_experiments"
path_images = join(path_data_experiments, fluid, "01_images_preprocessed")
path_save = join(path_data_experiments, fluid, "02_channel_edges")

# create directory for saving results and plots
os.makedirs(path_save, exist_ok=True)

In [ ]:
# distance of the start of vertical lines from the upper and lower edge, respectively (px)
# use 100 for experimental cases, 10 for simulation calibration cases.
start_y_samples_from_edge = 100

# length of the vertical lines (px)
# use 200 for experimental cases, 300 for simulation calibration cases.
length_sample_line = 200      

**Run the analysis**

In [ ]:
# list of first images per case
cases = [int(file) for file in os.listdir(path_images) if len(file)<3]

# initialize list for results
channel_edges = [[0]*(len(cases)), [0]*(len(cases))]

# loop over cases
for i in range(len(cases)):
    print(f"case {cases[i]}")

    # ---------------- get image
    
    # get last image of case to detect channel edges 
    img_name = os.listdir(join(path_images, str(cases[i])))[-1]

    # load image, convert it to grayscale
    img = cv2.imread(join(path_images, str(cases[i]), img_name))
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # plot the image
    plt.figure(figsize=(4,4))
    plt.imshow(img_gray, cmap='gray', vmin=0, vmax=255)

    # ---------------- detect channel edges

    # set start points for sampling
    x_samples = np.arange(10,len(img_gray[0]),10)
    start_ys = [start_y_samples_from_edge, len(img_gray)-start_y_samples_from_edge-length_sample_line]
    
    # loop over y start positions
    for yi in range(len(start_ys)):
        start_y = start_ys[yi]
        
        # initialize list for storing positions of darkest pixels along sampled lines
        list_y_min_gray = []

        # loop over x positions
        for xi in range(len(x_samples)):
            x_sample = x_samples[xi]
            (min_gray_val, y_min_gray_val) = find_min_along_vertical_line(img_gray, x_sample, start_y, length_sample_line)
            list_y_min_gray.append(y_min_gray_val)

        # get median
        y_median = np.median(list_y_min_gray)

        # save median in list
        channel_edges[yi][i] = y_median
        
        # ----------------plot
        
        # for a better overview, not all sample points are plotted
        plotevery=5

        # plot the sample lines
        for x in x_samples[::plotevery]:
            plt.plot([x,x], [start_y, start_y + length_sample_line], '--b', linewidth=1, 
                     label = "sampling lines" if  yi ==0 and x==x_samples[::plotevery][0] else None)
                                
        # plot the min grayscle values
        plt.plot(x_samples[::plotevery], list_y_min_gray[::plotevery], '+k', markersize=10, markeredgewidth=2, 
                 label='sampled points' if yi ==0 else None)

        # plot the median line    
        plt.plot([x_samples[0],x_samples[-1]], [y_median, y_median],'-r', linewidth=1,
                 label='median line' if yi ==0 else None)
                
        plt.legend(fancybox=False, framealpha=1.0, facecolor=[0.8,0.8,0.8], edgecolor='k', loc='center right', fontsize=10, 
                   bbox_to_anchor=(1.0,0.32), title=f"case {cases[i]}, img {img_name[-8:-4]}")

    # save plot
    #plt.savefig(join(path_save, str(cases[i])+".png"), bbox_inches='tight', dpi=600)
        

**save data**

In [ ]:
# create dataframe
df = pd.DataFrame(list(zip(channel_edges[0], channel_edges[1])),
               columns =['channel_edge_top', 'channel_edge_bottom'], index=(cases))

# save dataframe
df.to_csv(join(path_save, 'df_channel_edges.csv'), index_label='case')

# display dataframe
display(df)